In [1]:
import pickle
import pandas as pd
import os
import setup

import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm.notebook import tqdm, trange
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb
from torch.utils.data.sampler import SubsetRandomSampler
import pprint
import matplotlib.pyplot as plt
import torch

from sklearn.preprocessing import MinMaxScaler
import math
from torch.profiler import profile, record_function, ProfilerActivity

from operator import itemgetter
import operator
from random import randint
# from rnn_classes import Dog, DogInput, Race, Races, GRUNet, smallGRUNet, smalll_lin_GRUNet, smalll_prelin_GRUNet
import rnn_tools.rnn_classes as rnn_classes
from rnn_tools.raceDB import build_dataset
import importlib
import datetime
from rnn_tools.model_saver import model_saver, model_saver_wandb
# import rnn_tools.training_testing_gru as training_testing_gru
from torch.nn.utils.rnn import pack_padded_sequence, pack_sequence, pad_packed_sequence,pad_sequence, unpack_sequence, unpad_sequence
import rnn_tools.training_testing_gru_double as training_testing_gru_double
from goto_conversion import goto_conversion

import rnn_tools.training_testing_gru_extra_data as training_testing_gru_extra_data
import rnn_tools.training_testing_gru_extra_data_embedding as training_testing_gru_extra_data_embedding

c:\Users\Nick\Documents\GitHub\grvmodel\python


In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [3]:
torch.set_default_tensor_type(torch.FloatTensor)

c:\Users\Nick\.conda\envs\python311\Lib\site-packages\torch\__init__.py:614: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\tensor\python_tensor.cpp:453.)
  _C._set_default_tensor_type(t)


In [4]:
torch.__version__

'2.1.1'

In [5]:
# data_file = './data/topaz_data_w_bsp_new2.fth'
# df = pd.read_feather(data_file)
# df = df[df['state'].isin( ["NZ"])]
# df.to_feather('./data/topaz_data_w_bsp_new2_nz.fth')
# df.shape

In [6]:
os.getcwd()
import rnn_tools.raceDB
importlib.reload(rnn_tools.raceDB)
# os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")                                                                                                                                                                                                                                                                                                                
date = datetime.datetime.strptime("2019-01-01", "%Y-%m-%d").date()
hidden_size = 32
states = ["NZ","SA","WA"]
# states = ["VIC"]
# states = ["NSW", "VIC", "SA"]
states = ["NSW"]
# data_file = './data/gru_inputs_kitchen_sink_new_not_simple.fth'
# data_file = './data/gru_inputs_kitchen_sink_april.fth'
# data_file = './data/topaz_data_w_bsp_new_w_pred.fth'
data_file = './data/topaz_data_w_bsp_new7.fth'
# data_file = './data/gru_inputs_kitchen_sink_topaz.fth'
raceDB = rnn_tools.raceDB.build_dataset_topaz(data_file, hidden_size,state_filter=states,date_filter=date, margin_type='sftmin',v6=True,show_stats=False)
raceDB.create_new_weights_v2()

c:\Users\Nick\Documents\GitHub\grvmodel\python
(2518990, 196)
80
Latest date = 2024-05-19 00:00:00
size after state filter (2518990, 197)
(859808, 198)
Index(['index', 'meetingDate', 'state', 'track_name', 'dist', 'raceId',
       'race_grade', 'race_num', 'box', 'rugNumber',
       ...
       'prev_BSP_price', 'prev_start_price', 'stats_topaz', 'dogid', 'raceid',
       'stats_cols', 'track_hash', 'stats', 'race_time', 'stats_cuda'],
      dtype='object', length=199)
state  track_name       
NSW    Bathurst             16227
       Broken Hill           5372
       Bulli                18481
       Canberra                 1
       Casino               26099
       Coonabarabran          557
       Coonamble             3684
       Cowra                  596
       Dapto                21082
       Dubbo                21929
       Gosford              22927
       Goulburn             25520
       Grafton              24406
       Gunnedah             18287
       Kempsey            

100%|██████████| 25/25 [00:00<?, ?it/s]


race_ids=['397536999', '397537012', '397537000', '397537001', '397537003', '397537002', '397537004', '397537005', '397537007', '397537006', '397537010', '397537008', '397537009', '397537011', '397537020', '397537013', '397537015', '397537014', '397537016', '397537017', '397537018', '397537019', '397537022', '397537021', '397537031', '397537023', '397537027', '397624171', '397537025', '397537030', '397537026', '397537029', '397537024', '397624172', '397624174', '397624173', '397537032', '397537028', '397624165', '397584601', '397584596', '397624170', '397624169', '397624168', '397624167', '397624166', '397584595', '397584603', '397584597', '397584598', '397584599', '397584600', '397584602', '397584604', '397624343', '397537133', '397624350', '397624346', '397624349', '397537131', '397624342', '397624341', '397624351', '397624347', '397624344', '397584652', '397584651', '397584650', '397584649', '397584653', '397584635', '397537145', '397584634', '397624345', '397624348', '397537136', '3

In [7]:
def load_matching_state_dict(model, state_dict):
    model_dict = model.state_dict()

    # Filter out unnecessary keys
    matching_keys = {k: v for k, v in state_dict.items() if k in model_dict and v.size() == model_dict[k].size()}
    excluded_keys = set(state_dict.keys()) - set(matching_keys.keys())

    # Overwrite entries in the existing state dict
    model_dict.update(matching_keys)

    # Load the new state dict
    model.load_state_dict(model_dict)

    print("Loaded keys:")
    for key in matching_keys:
        print(key)

    print("\nExcluded keys:")
    for key in excluded_keys:
        print(key)
        

def model_pipeline(my_dataset=raceDB,config=None,prev_model=None, sweep=True, model_state_dict=None, prev_model_file=None, prev_model_version='450'):
    if my_dataset:
      dataset = my_dataset    
    else:
      dataset = raceDB
    # tell wandb to get started
    with wandb.init(project="NEW GRU V7 shuff", config=config,save_code=False):
      #  access all HPs through wandb.config, so logging matches execution!
      wandb.define_metric("loss_val", summary="min")
      wandb.define_metric("accuracy", summary="max")
      wandb.define_metric("ROI < 30", summary="max")
      wandb.define_metric("val_ROI < 30", summary="max")
      wandb.define_metric("relu roi", summary="max")
      
      config = wandb.config
      pprint.pprint(config)
      pprint.pprint(config.epochs)
      print(config)
      wandb.run.log_code('rnn_tools/')
      # input_size = raceDB.get_race_input([0,1])[0].full_input.shape[0] #create fix so messy
      print(config.input_type)

      for i in config.stat_list_dict.values():
        print(i)

      reg_stat_mask = []
      for stat,flag in config.stat_list_dict.items():
          stat_flag = flag
          reg_stat_mask = reg_stat_mask+[stat_flag]

      print(reg_stat_mask)
      stat_mask = [1]*27+53*[0]
      data_mask = [0]*26+[1]*20+[1]*20+[1]*20+[1]*20 # Reg, Dist, Box, T_box, T_dist
      data_mask = [1]*6+reg_stat_mask+[0]*20+[0]*20+[0]*20+[0]*20 # Reg, Dist, Box,T_box, T_dist
      data_mask =  [1]*27+53*[1]
      data_mask_size = sum(data_mask)
      print(f"{data_mask_size=}")
      stat_mask = torch.tensor(stat_mask).type(torch.bool).to(device)
      data_mask = torch.tensor(data_mask).type(torch.bool).to(device)
      # stat_mask = torch.ones_like(stat_mask).type(torch.bool).to(device)
      input_size = sum(stat_mask)

      print(stat_mask)
      print(stat_mask.shape)


      if 'batch_days' in config.keys() and not raceDB.batches_setup:
        pass
        raceDB.create_test_split_date(config['training_date_end'],val_date='2023-07-01')
        raceDB.create_dogs_test_split_date()
        raceDB.attach_races_to_dog_inputv2() 
        raceDB.reset_hidden()
        raceDB.create_batches(batch_days=config['batch_days'], end_date = config['training_date_end'], stat_mask=stat_mask,data_mask=data_mask,gen_packed_seq=True)
        raceDB.batches_setup = True
      if config['input_type'] == 'basic':
          print('here')
          raceDB.batches['packed_x'] = raceDB.batches['packed_x_basic']
          raceDB.batches['packed_y'] = raceDB.batches['packed_y_basic']
          raceDB.batches['packed_v'] = raceDB.batches['packed_v_basic']
          # input_size = raceDB.batches['packed_x'][0].data[0].shape[0]



      print(f"{input_size=}")
      config['stat_mask_tensor'] = torch.tensor(stat_mask, dtype=torch.uint8).to(device)     
      raceDB.reset_hidden(num_layers=config['num_layers'], hidden_size=config['hidden_size'])
      model = rnn_classes.GRUNetv3_extra_embedding(input_size,config['hidden_size'], num_layers=config['num_layers'],fc0_size=config['f0_layer_size'], fc1_size=config['f1_layer_size'],data_mask_size=data_mask_size)
      optimizer = optim.AdamW(model.parameters(), lr=config['learning_rate'])
      
      config['model_name'] = model.__class__.__name__
      if model_state_dict:
        model.load_state_dict(model_state_dict)
      if prev_model_file!=None:
        print(f"Loading model {prev_model_file}, version {prev_model_version}")
        model_name = prev_model_file
        model_loc = f"models/savedmodel/{model_name}/{model_name}_{prev_model_version}.pt"
        model_data = torch.load(model_loc,map_location=torch.device('cuda:0'))
        print(model_data['model_state_dict'].keys())

        load_matching_state_dict(model, model_data['model_state_dict'])
        config['parent model'] = prev_model_file
        hidden_state_init = model.h0.requires_grad_(True)
        raceDB.reset_hidden_w_param(hidden_state_init,num_layers=2, hidden_size=config['hidden_size'])
        raceDB.fill_hidden_states_dict_v2(model_data['db_train'])
        model = model.to(device)
        # raceDB.load_hidden_in_dict(hidden_in_dict=model_data['hidden_ins'],output_dict=model_data['output'])
        
        optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])
        # optimizer.load_state_dict(model_data['optim'])|
      else:
        optimizer = optim.RAdam(model.parameters(), lr=config['learning_rate'])

      raceDB.to_cuda()
      criterion = nn.CrossEntropyLoss(reduction='none')
      wandb.run.name = f"{' '.join(states)}-{wandb.run.name}"
      model = model.to(device)
      print(model)

      profit_model = rnn_classes.GRUNetv3_profit(raceDB).to('cuda:0')
      raceDB.profit_model = profit_model
      raceDB.reset_hidden_w_param(hidden_state_init,num_layers=2, hidden_size=config['hidden_size'])
      raceDB.dogsDict['nullDog'].input.hidden_out = (-torch.ones(config['hidden_size']+config['f1_layer_size'])).to('cuda:0')
      model = model.eval()
      training_testing_gru_extra_data_embedding.train_quick_pass(model,raceDB,config=config)
      # raceDB.reset_hidden_w_param(hidden_state_init,num_layers=2, hidden_size=config['hidden_size'])
      raceDB.dogsDict['nullDog'].input.hidden_out = (-torch.ones(config['hidden_size']+config['f1_layer_size'])).to('cuda:0')
      training_testing_gru_extra_data_embedding.test_model_v3(model,raceDB,criterion,config=config)
      # raceDB.reset_hidden_w_param(hidden_state_init,num_layers=2, hidden_size=config['hidden_size'])
      # raceDB.reset_hidden_w_param(hidden_state_init,num_layers=2, hidden_size=config['hidden_size'])
      # raceDB.dogsDict['nullDog'].input.hidden_out = (-torch.ones(config['hidden_size']+config['f1_layer_size'])).to('cuda:0')
      training_testing_gru_extra_data_embedding.validate_model_v3(model,raceDB,criterion,config=config)

      # try:
      #   return training_testing_gru_extra_data_embedding.train_double_v3(model, dataset, criterion, optimizer, 'na', config,update=True)
      # except (KeyboardInterrupt) as error:
      #   print(error)
      #   print("finished Early")
        
      # raceDB.create_hidden_states_dict_v2()
      # model_saver_wandb(model, optimizer, 450, 0.1, raceDB.hidden_states_dict_gru_v6,raceDB.train_hidden_dict , model_name="long nsw new  22000 RUN")


    return (model,dataset, optimizer)

In [8]:
raceDB.batches_setup = False

In [9]:
stat_list_dict = {'_dist_last__1':1,
    '_box_last__1':1,
    '_speed_avg_1':1,
    '_split_speed_avg_1':0,
    '_split_margin_avg_1':1,
    '_margin_avg_1':1,
    '_margin_time_avg_1':1,
    '_RunHomeTime_1':1,
    '_run_home_speed_1':1,
    '_first_out_avg_1':0,
    '_pos_out_avg_1':0,
    '_post_change_avg_1':0,
    '_races_1':1,
    '_wins_1':1,
    '_wins_last_1':1,
    '_weight_':1,
    '_min_time_':1,
    '_min_split_time_':1,
    '_last_start_price':0,
    '_last_start_prob':0,
}

wandb_config_static = {
    'hidden_size': 256,
    'stats': raceDB.stats_cols,
    'races': states,
    'datafile': data_file,
    'latest_date': raceDB.latest_date,
    'input_type': 'basic',
    'num_layers': 2,
    'batch_size': 750,
    'dropout': 0.3,
    'epochs': 2500,
    'learning_rate': 0.0001,
    'optimizer': 'adamW',
    'f0_layer_size': 128,
    'f1_layer_size': 64,
    'training_date_end': '2022-01-01',
    'notes':'GRU, with basic add on data looped in',
    'batch_days': 390,
    'stat_list_dict': stat_list_dict,
}

In [10]:
raceDB.reset_hidden(num_layers=wandb_config_static['num_layers'], hidden_size=wandb_config_static['hidden_size'])
torch.cuda.empty_cache()
raceDB.race_prices_to_prob()
print(raceDB.latest_date)
for race in raceDB.racesDict.values():
    race.loss = torch.tensor(1.0,requires_grad=True,device='cuda:0')  

for race in raceDB.racesDict.values():
    if race.classes.isnan().sum():
        print(race.classes.sum())
        print(race.raceid)
        race.classes = race.classes.nan_to_num(0,0,0)

2024-05-18 00:00:00


In [11]:
importlib.reload(rnn_tools.rnn_classes)
importlib.reload(rnn_classes)
importlib.reload(training_testing_gru_extra_data)
importlib.reload(training_testing_gru_extra_data_embedding)
# (model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static,sweep=False)
# model_pipeline(raceDB,config=wandb_config_static,sweep=False, prev_model_file='VIC SA NSW-ancient-valley-726', prev_model_version = 280)
# model_pipeline(raceDB,config=wandb_config_static,sweep=False, prev_model_file='VIC-dandy-wood-653', prev_model_version = 40)
model_pipeline(raceDB,config=wandb_config_static,sweep=False,prev_model_file='NSW-hardy-snowflake-834', prev_model_version = 200)
# (model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static,sweep=False)
# all_price_df = model_pipeline(raceDB,config=wandb_config_static,sweep=False)
# ll

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nickojelly. Use `wandb login --relogin` to force relogin


{'hidden_size': 256, 'stats': "['dogAgeScaled', 'boxNumber', 'incomingGrade_val', 'weightInKgScaled', 'hasEntryBoxNumberPlus1', 'hasEntryBoxNumberMinus1', 'rolling_box_win_percentage', 'dog_incomingGrade_val_mean_1', 'dog_startPrice_mean_1', 'dog_distance_mean_1', 'dog_boxNumber_mean_1', 'dog_runTimeNorm_mean_1', 'dog_win_runTimeNorm_mean_1', 'dog_delta_win_runTimeNorm_mean_1', 'dog_place_mean_1', 'dog_resultMargin_mean_1', 'dog_finishingPlaceMovement_mean_1', 'dog_StartingPlaceMovement_mean_1', 'dog_split_time_margin_mean_1', 'dog_split_runTimeNorm_mean_1', 'dog_time_1_mean_1', 'dog_run_home_TimeNorm_mean_1', 'dog_performance_to_price_mean_1', 'dog_averageSpeed_mean_1', 'dog_win_mean_1', 'dog_time_1_mean_365D', 'trainer_incomingGrade_val_mean_365D', 'trainer_startPrice_mean_365D', 'trainer_distance_mean_365D', 'trainer_boxNumber_mean_365D', 'trainer_runTimeNorm_mean_365D', 'trainer_win_runTimeNorm_mean_365D', 'trainer_delta_win_runTimeNorm_mean_365D', 'trainer_place_mean_365D', 'train

100%|██████████| 22213/22213 [00:17<00:00, 1278.95it/s]


[(datetime.date(2019, 1, 2), datetime.date(2020, 1, 27)), (datetime.date(2020, 1, 27), datetime.date(2021, 2, 20)), (datetime.date(2021, 2, 20), datetime.date(2021, 12, 31))]
2020-01-27
2021-02-20
2021-12-31
Train examples [13280, 12301, 10876]


100%|██████████| 22212/22212 [00:25<00:00, 885.44it/s]
3it [01:21, 27.04s/it]


...

100%|██████████| 22213/22213 [00:00<00:00, 770251.45it/s]

filled =3912
empty  =18301
0.17611308693107638null_dog=0


GRUNetv3_extra_embedding(
  (gru): GRU(27, 256, num_layers=2, dropout=0.3)
  (relu): ReLU()
  (fc0): Linear(in_features=2611, out_features=2048, bias=True)
  (batch_norm): BatchNorm1d(27, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batch_norm_data): BatchNorm1d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (track_embedding): Embedding(1024, 50)
  (extra_1): GRUNetv3_simple_extra_data(
    (batch_norm): LazyBatchNorm1d(0, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
    (fc0_p1): Linear(in_features=80, out_features=128, bias=True)
    (fc0_p1_drop): Dropout(p=0.3, inplace=False)
    (fc0_p2): Linear(in_features=128, out_features=64, bias=True)
    (fc0_p2_drop): Dropout(p=0.3, inplace=False)
    (fc0_p3): Linear(in_features=64, out_features=64, bias=True)
    (fc0_p3_drop): Dropout(p=0.3, inplace=False)
  )
  (relu0): ReLU()
  (drop1): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_features=2048, out_

ROI < 30,▁▁
ROI < 30 2,▁▁
accuracy,▁▁
accuracy2,▁
avg_win_price,▁▁
epoch,▁▁▁
favorite_accuracy,▁▁
flat_simple,▁▁
kelly roi,▁▁
kelly seq roi,▁▁
kelly seq roi < 30,▁▁


(GRUNetv3_extra_embedding(
   (gru): GRU(27, 256, num_layers=2, dropout=0.3)
   (relu): ReLU()
   (fc0): Linear(in_features=2611, out_features=2048, bias=True)
   (batch_norm): BatchNorm1d(27, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (batch_norm_data): BatchNorm1d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (track_embedding): Embedding(1024, 50)
   (extra_1): GRUNetv3_simple_extra_data(
     (batch_norm): LazyBatchNorm1d(0, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (relu): ReLU()
     (fc0_p1): Linear(in_features=80, out_features=128, bias=True)
     (fc0_p1_drop): Dropout(p=0.3, inplace=False)
     (fc0_p2): Linear(in_features=128, out_features=64, bias=True)
     (fc0_p2_drop): Dropout(p=0.3, inplace=False)
     (fc0_p3): Linear(in_features=64, out_features=64, bias=True)
     (fc0_p3_drop): Dropout(p=0.3, inplace=False)
   )
   (relu0): ReLU()
   (drop1): Dropout(p=0.3, inplace=False)
   (fc1): Linear(in

In [12]:
raceDB.stats_cols = eval(raceDB.stats_cols)

In [13]:
raceDB.stats_cols

['dogAgeScaled',
 'boxNumber',
 'incomingGrade_val',
 'weightInKgScaled',
 'hasEntryBoxNumberPlus1',
 'hasEntryBoxNumberMinus1',
 'rolling_box_win_percentage',
 'dog_incomingGrade_val_mean_1',
 'dog_startPrice_mean_1',
 'dog_distance_mean_1',
 'dog_boxNumber_mean_1',
 'dog_runTimeNorm_mean_1',
 'dog_win_runTimeNorm_mean_1',
 'dog_delta_win_runTimeNorm_mean_1',
 'dog_place_mean_1',
 'dog_resultMargin_mean_1',
 'dog_finishingPlaceMovement_mean_1',
 'dog_StartingPlaceMovement_mean_1',
 'dog_split_time_margin_mean_1',
 'dog_split_runTimeNorm_mean_1',
 'dog_time_1_mean_1',
 'dog_run_home_TimeNorm_mean_1',
 'dog_performance_to_price_mean_1',
 'dog_averageSpeed_mean_1',
 'dog_win_mean_1',
 'dog_time_1_mean_365D',
 'trainer_incomingGrade_val_mean_365D',
 'trainer_startPrice_mean_365D',
 'trainer_distance_mean_365D',
 'trainer_boxNumber_mean_365D',
 'trainer_runTimeNorm_mean_365D',
 'trainer_win_runTimeNorm_mean_365D',
 'trainer_delta_win_runTimeNorm_mean_365D',
 'trainer_place_mean_365D',
 'tr

In [15]:
stat_cols= ['dogAgeScaled',
 'boxNumber',
 'incomingGrade_val',
 'weightInKgScaled',
 'hasEntryBoxNumberPlus1',
 'hasEntryBoxNumberMinus1',
 'rolling_box_win_percentage',
 'dog_incomingGrade_val_mean_1',
 'dog_startPrice_mean_1',
 'dog_distance_mean_1',
 'dog_boxNumber_mean_1',
 'dog_runTimeNorm_mean_1',
 'dog_win_runTimeNorm_mean_1',
 'dog_delta_win_runTimeNorm_mean_1',
 'dog_place_mean_1',
 'dog_resultMargin_mean_1',
 'dog_finishingPlaceMovement_mean_1',
 'dog_StartingPlaceMovement_mean_1',
 'dog_split_time_margin_mean_1',
 'dog_split_runTimeNorm_mean_1',
 'dog_time_1_mean_1',
 'dog_run_home_TimeNorm_mean_1',
 'dog_performance_to_price_mean_1',
 'dog_averageSpeed_mean_1',
 'dog_win_mean_1',]
wandb_config_static['variable'] = 'No change'
wandb_config_static['shuffle_type'] = 'both'
model_pipeline(raceDB,config=wandb_config_static,sweep=False,prev_model_file='NSW-usual-snowflake-820', prev_model_version = 280)
original_pack = raceDB.batches['packed_y_basic']
original_data = raceDB.packed_y_data
for i in range(0,len(stat_cols)):
    print(f"{stat_cols[i]=}")
    wandb_config_static['variable'] = stat_cols[i]

    raceDB.batches['packed_y_basic'] = original_pack
    raceDB.batches['packed_y_data'] = original_data
    x = original_pack.data.to('cpu')
    idx = torch.randperm(x.size(0))
    x[:, i] = x[idx,i] 
    data = original_data.data.to('cpu')
    data[:,i] = data[idx,i]
    raceDB.batches['packed_y_basic'] = raceDB.batches['packed_y_basic']._replace(data=x.to('cuda:0'))
    raceDB.packed_y_data = original_data = raceDB.packed_y_data._replace(data=data.to('cuda:0'))
    model_pipeline(raceDB,config=wandb_config_static,sweep=False,prev_model_file='NSW-hardy-snowflake-834', prev_model_version = 200)

{'hidden_size': 256, 'stats': "['dogAgeScaled', 'boxNumber', 'incomingGrade_val', 'weightInKgScaled', 'hasEntryBoxNumberPlus1', 'hasEntryBoxNumberMinus1', 'rolling_box_win_percentage', 'dog_incomingGrade_val_mean_1', 'dog_startPrice_mean_1', 'dog_distance_mean_1', 'dog_boxNumber_mean_1', 'dog_runTimeNorm_mean_1', 'dog_win_runTimeNorm_mean_1', 'dog_delta_win_runTimeNorm_mean_1', 'dog_place_mean_1', 'dog_resultMargin_mean_1', 'dog_finishingPlaceMovement_mean_1', 'dog_StartingPlaceMovement_mean_1', 'dog_split_time_margin_mean_1', 'dog_split_runTimeNorm_mean_1', 'dog_time_1_mean_1', 'dog_run_home_TimeNorm_mean_1', 'dog_performance_to_price_mean_1', 'dog_averageSpeed_mean_1', 'dog_win_mean_1', 'dog_time_1_mean_365D', 'trainer_incomingGrade_val_mean_365D', 'trainer_startPrice_mean_365D', 'trainer_distance_mean_365D', 'trainer_boxNumber_mean_365D', 'trainer_runTimeNorm_mean_365D', 'trainer_win_runTimeNorm_mean_365D', 'trainer_delta_win_runTimeNorm_mean_365D', 'trainer_place_mean_365D', 'train

C:\Users\Nick\AppData\Local\Temp\ipykernel_22660\344181034.py:87: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  config['stat_mask_tensor'] = torch.tensor(stat_mask, dtype=torch.uint8).to(device)


Loading model NSW-usual-snowflake-820, version 280
odict_keys(['h0', 'gru.weight_ih_l0', 'gru.weight_hh_l0', 'gru.bias_ih_l0', 'gru.bias_hh_l0', 'gru.weight_ih_l1', 'gru.weight_hh_l1', 'gru.bias_ih_l1', 'gru.bias_hh_l1', 'fc0.weight', 'fc0.bias', 'batch_norm.weight', 'batch_norm.bias', 'batch_norm.running_mean', 'batch_norm.running_var', 'batch_norm.num_batches_tracked', 'batch_norm_data.weight', 'batch_norm_data.bias', 'batch_norm_data.running_mean', 'batch_norm_data.running_var', 'batch_norm_data.num_batches_tracked', 'track_embedding.weight', 'extra_1.batch_norm.weight', 'extra_1.batch_norm.bias', 'extra_1.batch_norm.running_mean', 'extra_1.batch_norm.running_var', 'extra_1.batch_norm.num_batches_tracked', 'extra_1.fc0_p1.weight', 'extra_1.fc0_p1.bias', 'extra_1.fc0_p2.weight', 'extra_1.fc0_p2.bias', 'extra_1.fc0_p3.weight', 'extra_1.fc0_p3.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias', 'fc3.weight', 'fc3.bias'])
Loaded keys:
h0
gru.weight_hh_l0
gru.bias_ih_l0
gru.bias_h

100%|██████████| 22213/22213 [00:00<00:00, 990064.87it/s]

filled =3912
empty  =18301
0.17611308693107638null_dog=0


...

Loading model NSW-hardy-snowflake-834, version 200
odict_keys(['h0', 'gru.weight_ih_l0', 'gru.weight_hh_l0', 'gru.bias_ih_l0', 'gru.bias_hh_l0', 'gru.weight_ih_l1', 'gru.weight_hh_l1', 'gru.bias_ih_l1', 'gru.bias_hh_l1', 'fc0.weight', 'fc0.bias', 'batch_norm.weight', 'batch_norm.bias', 'batch_norm.running_mean', 'batch_norm.running_var', 'batch_norm.num_batches_tracked', 'batch_norm_data.weight', 'batch_norm_data.bias', 'batch_norm_data.running_mean', 'batch_norm_data.running_var', 'batch_norm_data.num_batches_tracked', 'track_embedding.weight', 'extra_1.batch_norm.weight', 'extra_1.batch_norm.bias', 'extra_1.batch_norm.running_mean', 'extra_1.batch_norm.running_var', 'extra_1.batch_norm.num_batches_tracked', 'extra_1.fc0_p1.weight', 'extra_1.fc0_p1.bias', 'extra_1.fc0_p2.weight', 'extra_1.fc0_p2.bias', 'extra_1.fc0_p3.weight', 'extra_1.fc0_p3.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias', 'fc3.weight', 'fc3.bias'])
Loaded keys:
h0
gru.weight_ih_l0
gru.weight_hh_l0
gru.bias

100%|██████████| 22213/22213 [00:00<00:00, 965712.45it/s]

filled =3912
empty  =18301
0.17611308693107638null_dog=0


GRUNetv3_extra_embedding(
  (gru): GRU(27, 256, num_layers=2, dropout=0.3)
  (relu): ReLU()
  (fc0): Linear(in_features=2611, out_features=2048, bias=True)
  (batch_norm): BatchNorm1d(27, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batch_norm_data): BatchNorm1d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (track_embedding): Embedding(1024, 50)
  (extra_1): GRUNetv3_simple_extra_data(
    (batch_norm): LazyBatchNorm1d(0, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
    (fc0_p1): Linear(in_features=80, out_features=128, bias=True)
    (fc0_p1_drop): Dropout(p=0.3, inplace=False)
    (fc0_p2): Linear(in_features=128, out_features=64, bias=True)
    (fc0_p2_drop): Dropout(p=0.3, inplace=False)
    (fc0_p3): Linear(in_features=64, out_features=64, bias=True)
    (fc0_p3_drop): Dropout(p=0.3, inplace=False)
  )
  (relu0): ReLU()
  (drop1): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_features=2048, out_

ROI < 30,▁▁
ROI < 30 2,▁▁
accuracy,▁▁
accuracy2,▁
avg_win_price,▁▁
epoch,▁▁▁
favorite_accuracy,▁▁
flat_simple,▁▁
kelly roi,▁▁
kelly seq roi,▁▁
kelly seq roi < 30,▁▁


In [ ]:
lprev_model_file='NZ-glad-river-548'
prev_model_version = 485
model_name = prev_model_file    
model_loc = f"models/savedmodel/{model_name}/{model_name}_{prev_model_version}.pt"
model_data = torch.load(model_loc,map_location=torch.device('cuda:0'))
print(model_data['model_state_dict'].keys())

NameError: name 'prev_model_file' is not defined

In [ ]:
print(model_data['model_state_dict']['extra_1.fc0_p1.weight'].shape)

torch.Size([128, 54])


In [ ]:
stat_mask = [1]*18+36*[0]
data_mask =  [0]*18+36*[1]
data_mask_size = sum(data_mask)
print(f"{data_mask_size=}")
stat_mask = torch.tensor(stat_mask).type(torch.bool).to(device)
data_mask = torch.tensor(data_mask).type(torch.bool).to(device)
input_size = sum(stat_mask)

data_mask_size=36


In [ ]:
config = wandb_config_static
# input_size = model_data['input_size']
model = rnn_classes.GRUNetv3_extra_embedding(input_size,config['hidden_size'], num_layers=config['num_layers'],fc0_size=config['f0_layer_size'], fc1_size=config['f1_layer_size'],data_mask_size=data_mask_size)

c:\Users\Nick\.conda\envs\python311\Lib\site-packages\torch\nn\modules\lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [ ]:
model

GRUNetv3_extra_embedding(
  (gru): GRU(18, 256, num_layers=2, dropout=0.3)
  (relu): ReLU()
  (fc0): Linear(in_features=2611, out_features=2048, bias=True)
  (batch_norm): BatchNorm1d(18, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batch_norm_data): BatchNorm1d(36, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (track_embedding): Embedding(1024, 50)
  (extra_1): GRUNetv3_simple_extra_data(
    (batch_norm): LazyBatchNorm1d(0, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
    (fc0_p1): Linear(in_features=36, out_features=128, bias=True)
    (fc0_p1_drop): Dropout(p=0.3, inplace=False)
    (fc0_p2): Linear(in_features=128, out_features=64, bias=True)
    (fc0_p2_drop): Dropout(p=0.3, inplace=False)
    (fc0_p3): Linear(in_features=64, out_features=64, bias=True)
    (fc0_p3_drop): Dropout(p=0.3, inplace=False)
  )
  (relu0): ReLU()
  (drop1): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_features=2048, out_

In [ ]:
for i in raceDB.hidden_state_inits:
    print(i[0,0])

AttributeError: 'Races' object has no attribute 'hidden_state_inits'

In [ ]:
hidden_state_init = model.h0
raceDB.reset_hidden_w_param(hidden_state_init,num_layers=2, hidden_size=256)

In [ ]:
raceDB.dogsDict['408750377'].hidden